# Objetivo do Projeto:

Tratamento de Dados: Realizar a junção (JOIN) de duas tabelas utilizando SQL para consolidar as informações. Análise de Dados: Exportar os dados resultantes para um arquivo CSV. Visualização de Dados: Desenvolver um dashboard interativo e informativo para visualização das principais métricas e insights do e-commerce.

**Tabelas Disponibilizadas:**

**Tabela de Transações:** Contém os registros de transações realizadas pelos clientes, incluindo detalhes como ID da transação, valor e outros.


**Tabela de Dados Pessoais:** Contém as informações pessoais dos clientes, como ID do cliente, nome, genero, cidade, etc.

**Chave de Ligação:** As tabelas se relacionam através da coluna ID_CLIENT, que é a chave identificadora dos clientes.

In [2]:
import sqlite3
import pandas as pd
import numpy as np

**Vamos carregar a base de dados e realizar alguns ajustes antes de dar join nas tabelas** 

In [4]:
df_transacoes = pd.read_csv("TB_TRANSACOES_PROJETO_ECOMM.csv", delimiter=';')
df_clientes = pd.read_csv("TB_CLIENTES_PROJETO_ECOMM.csv", delimiter=';')

In [5]:
print(df_transacoes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id_client  367 non-null    int64 
 1   Category   367 non-null    object
 2   Price      367 non-null    object
 3   Card Type  367 non-null    object
dtypes: int64(1), object(3)
memory usage: 11.6+ KB
None


In [6]:
print(df_clientes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   state_name  175 non-null    object
 1   First_name  175 non-null    object
 2   Gender      175 non-null    object
 3   Job_Title   175 non-null    object
 4   Id_client   175 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 7.0+ KB
None


**Abaixo vamos transformar as variáveis categórias em mínusculo para não haver conflitos**

In [8]:
df_transacoes.columns = df_transacoes.columns.str.lower()
df_clientes.columns = df_clientes.columns.str.lower()

In [10]:
df_transacoes = df_transacoes.rename(columns={'card type': 'card_type'})

In [11]:
df_transacoes['card_type'].unique()

array(['mastercard'], dtype=object)

**Agora veremos a relação entre as duas tabelas, isto é: quantos clientes com IDs cadastrados fizeram transações e quantos não fizeram transações. 
Além disso vamos analisar a quantidade de transações feitas sem um ID cadastrado.**

In [12]:
clientes_transacoes = df_transacoes['id_client'].unique()
clientes_cadastrados = df_clientes['id_client'].unique()

clientes_inexistentes = set(clientes_transacoes) - set(clientes_cadastrados)
print(f" Total de IDs únicos em transações: {len(clientes_transacoes)}")
print(f" Total de IDs únicos cadastrados: {len(clientes_cadastrados)}")
print(f" Clientes em transações NÃO cadastrados: {len(clientes_inexistentes)}")
print(f" Clientes em transações CADASTRADOS: {len(set(clientes_transacoes) & set(clientes_cadastrados))}")

 Total de IDs únicos em transações: 241
 Total de IDs únicos cadastrados: 175
 Clientes em transações NÃO cadastrados: 71
 Clientes em transações CADASTRADOS: 170


In [13]:
clientes_sem_transacoes = set(clientes_cadastrados) - set(clientes_transacoes)
print(f"\n📊 Clientes CADASTRADOS que NUNCA fizeram transações: {len(clientes_sem_transacoes)}")
if clientes_sem_transacoes:
    print(f"IDs: {sorted(list(clientes_sem_transacoes))}")


📊 Clientes CADASTRADOS que NUNCA fizeram transações: 5
IDs: [np.int64(80), np.int64(81), np.int64(82), np.int64(83), np.int64(84)]


**Vamos carregar os banco de dados no SQL para poder criar uma query e juntar as tabelas com as variáveis que precisamos.**

In [14]:
conn = sqlite3.connect('projeto.db')
# Carregar o DataFrame no banco de dados SQLite - criando tb_transacoes e tb_clientes
df_transacoes.to_sql('tb_transacoes', conn, index=False, if_exists='replace')
df_clientes.to_sql('tb_clientes', conn, index=False, if_exists='replace')

175

In [15]:
def run_query(query):
    return pd.read_sql_query(query, conn)

**Como todas as transações foram feitas pelo cartão mastercard, eliminamos essa variável. Também eliminamos o nome das pessoas porque é irrelevante para o estudo.
Pela alta quantidade de profissões distintas, optamos por não incluí-las no dataset final também.**

**Foi feito um left join basicamente porque havia 5 ID's de clientes que não fizeram transação alguma, isso é facilmente descrito em algum relatório, mas para levantar os gráficos, não há relevância já que é uma quantidade ínfima de clientes.**

In [16]:
query = """
SELECT
A.id_client,
A.category,
A.price,
B.state_name,
B.gender
FROM tb_transacoes AS A
LEFT JOIN tb_clientes as B
ON A.id_client = B.id_client
"""
result_df = run_query(query)
print(result_df)

     id_client     category   price state_name       gender
0           37  Electronics   72,93         ND  Genderqueer
1           38      Jewelry  121,89         PA         Male
2           39         Baby    64,3         MA       Female
3           40     Outdoors    9,48         OR       Female
4            5     Outdoors   61,95         MN       Female
..         ...          ...     ...        ...          ...
362        363      Jewelry   44,32       None         None
363        364         Kids  120,42       None         None
364        365      Grocery   24,31       None         None
365        366        Tools  111,53       None         None
366        367    Computers   84,57       None         None

[367 rows x 5 columns]


**Vamos criar uma categoria para gênero e estado de quem não preencheu, sendo as respostas 'unknown' (desconhecido) e 'not specified' (não específicado). Por fim salvaremos o dataset final e criaremos um dashboard para análises** 

In [17]:
nulos_state = result_df['state_name'].isnull().sum()
nulos_gender = result_df['gender'].isnull().sum()

print(f"state_name nulos: {nulos_state} ({nulos_state/len(result_df)*100:.1f}%)")
print(f"gender nulos: {nulos_gender} ({nulos_gender/len(result_df)*100:.1f}%)")

state_name nulos: 71 (19.3%)
gender nulos: 71 (19.3%)


In [21]:
result_df['gender'] = result_df['gender'].fillna('unknown')
result_df['state_name'] = result_df['state_name'].fillna('not specified')

In [19]:
result_df.to_csv('dados_ecommerce_final.csv', index=False)